In [ ]:
import requests
from pprint import pprint
import xml.etree.ElementTree as ET

def make_request(url, method='GET', data=None, headers=None):

    if headers is None:
        headers = {}

    if method.upper() == 'GET':
        response = requests.get(url, headers=headers)
    elif method.upper() == 'POST':
        response = requests.post(url, data=data, headers=headers)
    else:
        raise ValueError("Unsupported HTTP method: {}".format(method))

    return response


# https://nrid.nii.ac.jp/opensearch/?(パラメータ=値)& (パラメータ=値)&…&(パラメータ=値)
def build_url(base_url, **params):
    query_string = '&'.join(f"{key}={value}" for key, value in params.items())
    return f"{base_url}?{query_string}" if query_string else base_url

def fetch_data(base_url, **params):
    url = build_url(base_url, **params)
    response = make_request(url)

    if response.status_code == 200:
        return response
    else:
        response.raise_for_status()  # Raise an error for bad responses

def parse_xml_response(xml_content):
    """XMLレスポンスをパースしてPythonオブジェクトに変換する"""
    root = ET.fromstring(xml_content)
    return root

# メインロジックを実行
base_url = "https://nrid.nii.ac.jp/opensearch/"
params = {
    'kw': 'ディープラーニング',
    'appid': 'gu782hFEjCChFcYxyeb9'
}

data = fetch_data(base_url, **params)

# レスポンスの内容を確認
print("レスポンスのContent-Type:", data.headers.get('Content-Type'))
print("レスポンスの先頭1000文字:")
print(data._content[:1000])

try:
    # XMLとして解析を試みる
    xml_root = parse_xml_response(data._content)
    print("XMLとしての解析に成功")
except ET.ParseError as e:
    print(f"XMLとしての解析に失敗: {e}")

    # UTF-8でデコードしてから解析を試みる
    try:
        xml_content_decoded = data._content.decode('utf-8')
        xml_root = parse_xml_response(xml_content_decoded)
        print("デコード後のXMLとしての解析に成功")
    except Exception as e:
        print(f"デコード後の解析にも失敗: {e}")
else:
    # XMLの内容を表示
    print("XMLのルートタグ:", xml_root.tag)
    print("XMLのルート属性:", xml_root.attrib)


レスポンスのContent-Type: text/html; charset=UTF-8
レスポンスの先頭1000文字:
b'<!DOCTYPE html>\n<html>\n  <head>\n   \n    <!-- meta -->\n    <meta charset="UTF-8">\n    <meta http-equiv="Pragma" content="no-cache" />\n    <meta http-equiv="Cache-Control" content="no-cache" />\n    <meta http-equiv="Cache-Control" content="no-store" />\n    <meta http-equiv="Cache-Control" content="must-revalidate" />\n    <meta http-equiv="Expires" content="0" />\n    \n    <meta name="ROBOTS" content="NONE" />\n    <meta name="ROBOTS" content="NOINDEX,NOFOLLOW" />\n    <meta name="ROBOTS" content="NOARCHIVE" />\n    \n    <link rel="shortcut icon" href="/static/images/favicon_kakenhi.ico" type="image/vnd.microsoft.ico" />\n    <link rel="icon" href="/static/images/favicon_kakenhi.ico" type="image/vnd.microsoft.ico" />\n    <!-- Bootstrap core CSS -->\n    <link id="switch_style" href="/static/css/bootstrap-modal.css?1691548633" rel="stylesheet" />\n    <!-- Custom styles for this template -->\n    <link href="/stati

In [17]:
pprint(data._content)
# xmlとして保存
with open('response.xml', 'wb') as f:
    f.write(data._content)


(b'<!DOCTYPE html>\n<html>\n  <head>\n   \n    <!-- meta -->\n    <meta char'
 b'set="UTF-8">\n    <meta http-equiv="Pragma" content="no-cache" />\n    <me'
 b'ta http-equiv="Cache-Control" content="no-cache" />\n    <meta http-equiv'
 b'="Cache-Control" content="no-store" />\n    <meta http-equiv="Cache-Contr'
 b'ol" content="must-revalidate" />\n    <meta http-equiv="Expires" content='
 b'"0" />\n    \n    <meta name="ROBOTS" content="NONE" />\n    <meta name="RO'
 b'BOTS" content="NOINDEX,NOFOLLOW" />\n    <meta name="ROBOTS" content="NOA'
 b'RCHIVE" />\n    \n    <link rel="shortcut icon" href="/static/images/favic'
 b'on_kakenhi.ico" type="image/vnd.microsoft.ico" />\n    <link rel="icon" h'
 b'ref="/static/images/favicon_kakenhi.ico" type="image/vnd.microsoft.ico" '
 b'/>\n    <!-- Bootstrap core CSS -->\n    <link id="switch_style" href="/st'
 b'atic/css/bootstrap-modal.css?1691548633" rel="stylesheet" />\n    <!-- Cu'
 b'stom styles for this template -->\n    <link href="/stati

In [17]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

def make_request(url, method='GET', data=None, headers=None):
    if headers is None:
        headers = {}

    if method.upper() == 'GET':
        response = requests.get(url, headers=headers)
    elif method.upper() == 'POST':
        response = requests.post(url, data=data, headers=headers)
    else:
        raise ValueError("Unsupported HTTP method: {}".format(method))

    return response

def build_url(base_url, **params):
    query_string = '&'.join(f"{key}={value}" for key, value in params.items())
    return f"{base_url}?{query_string}" if query_string else base_url

def fetch_data(base_url, **params):
    url = build_url(base_url, **params)
    response = make_request(url)

    if response.status_code == 200:
        return response
    else:
        response.raise_for_status()  # Raise an error for bad responses

def parse_html_response(html_content):
    """HTMLレスポンスをパースしてBeautifulSoupオブジェクトに変換する"""
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

def get_search_results(soup):
    """検索結果を抽出する"""
    results = []

    # 研究者リストの各アイテムを取得
    researchers = soup.select('.listitem.xfolkentry')

    for researcher in researchers:
        # 基本情報の取得
        name_elem = researcher.select_one('.item_mainTitle a')
        if not name_elem:
            continue

        name = name_elem.text.strip()
        # ID部分を取得 (例: (70213486))
        researcher_id = None
        id_match = None
        if '(' in name and ')' in name:
            id_match = name.split('(')[-1].split(')')[0]
            if id_match.isdigit():
                researcher_id = id_match

        # プロフィールURL
        profile_url = name_elem.get('href', '')

        # 所属情報の取得
        affiliation = []
        affiliation_elem = researcher.select_one('tr:has(th:contains("所属")) td')
        if affiliation_elem:
            for line in affiliation_elem.stripped_strings:
                if line.strip():
                    affiliation.append(line.strip())

        # キーワードの取得
        keywords = []
        keywords_elem = researcher.select_one('tr:has(th:contains("キーワード")) td')
        if keywords_elem:
            keywords = [a.text.strip() for a in keywords_elem.select('a')]

        # 研究課題数と研究成果数
        research_projects = 0
        research_projects_elem = researcher.select_one('tr:has(th:contains("研究課題数")) td')
        if research_projects_elem:
            try:
                research_projects = int(research_projects_elem.text.strip().replace(',', ''))
            except:
                pass

        research_results = 0
        research_results_elem = researcher.select_one('tr:has(th:contains("研究成果数")) td')
        if research_results_elem:
            try:
                research_results = int(research_results_elem.text.strip().replace(',', ''))
            except:
                pass

        # 結果を辞書として格納
        researcher_info = {
            'name': name.split('(')[0].strip() if '(' in name else name,
            'researcher_id': researcher_id,
            'profile_url': profile_url,
            'affiliation': affiliation,
            'keywords': keywords,
            'research_projects': research_projects,
            'research_results': research_results
        }

        results.append(researcher_info)

    return results

# メインロジックを実行
base_url = "https://nrid.nii.ac.jp/opensearch/"
params = {
    'rw': 500,
    'qf': '深層学習 OR 人工知能 OR 機械学習',
    'qd': '[審査区分:審査区分]大区分J OR [審査区分:審査希望区分(新学術領域研究)]複合領域',
    'appid': 'gu782hFEjCChFcYxyeb9',
    'od': '1',
}

data = fetch_data(base_url, **params)
soup = parse_html_response(data.content)

# 検索結果の総数を取得
total_results_elem = soup.select_one('.search-term-selected')
total_results = "不明"
if total_results_elem:
    text = total_results_elem.text
    import re
    match = re.search(r'(\d+,?\d*)件', text)
    if match:
        total_results = match.group(1)

print(f"検索結果の総数: {total_results}")

# 検索結果の抽出
results = get_search_results(soup)
print(f"取得した研究者数: {len(results)}")

# 結果の表示（最初の3件だけ表示）
for i, researcher in enumerate(results):
    print(f"\n--- 研究者 {i+1} ---")
    print(f"氏名: {researcher['name']}")
    print(f"研究者ID: {researcher['researcher_id']}")
    print(f"所属: {', '.join(researcher['affiliation'][:1])}")  # 最初の所属だけ表示
    print(f"キーワード: {', '.join(researcher['keywords'][:5])}")  # 最初の5キーワードだけ表示
    print(f"研究課題数: {researcher['research_projects']}")
    print(f"研究成果数: {researcher['research_results']}")
    print(f"プロフィールURL: {researcher['profile_url']}")

検索結果の総数: 640
取得した研究者数: 500

--- 研究者 1 ---
氏名: 森岡 博史  MORIOKA Hiroshi
研究者ID: 20739552
所属: 滋賀大学, データサイエンス学系, 准教授
キーワード: 機械学習, 深層学習, 非線形解析, 因果探索, 計算神経科学
研究課題数: 6
研究成果数: 23
プロフィールURL: /nrid/1000020739552/

--- 研究者 2 ---
氏名: 山口 光太  ヤマグチ コウタ
研究者ID: 10742596
所属: 東北大学, 情報科学研究科, 助教
キーワード: コンピュータビジョン, 画像認識, 深層学習, 機械学習, 人工知能
研究課題数: 2
研究成果数: 0
プロフィールURL: /nrid/1000010742596/

--- 研究者 3 ---
氏名: 中島 悠太  Nakashima Yuta
研究者ID: 70633551
所属: 大阪大学, 産業科学研究所, 教授
キーワード: 深層学習, 人工知能, 重要領域推定, 映像要約, 仏像
研究課題数: 13
研究成果数: 86
プロフィールURL: /nrid/1000070633551/

--- 研究者 4 ---
氏名: 中山 浩太郎  Nakayama Kotaro
研究者ID: 00512097
所属: 東京大学, 大学院工学系研究科(工学部), 学術支援専門職員
キーワード: 人工知能, 情報システム, 情報検索, スケーラビリティ, Deep Learning
研究課題数: 4
研究成果数: 102
プロフィールURL: /nrid/1000000512097/

--- 研究者 5 ---
氏名: 塚田 武志  Tsukada Takeshi
研究者ID: 50758951
所属: 千葉大学, 大学院理学研究院, 教授
キーワード: プログラム検証, 機械学習, 線形論理, π計算, ゲーム意味論
研究課題数: 6
研究成果数: 24
プロフィールURL: /nrid/1000050758951/

--- 研究者 6 ---
氏名: 関山 太朗  Sekiyama Taro
研究者ID: 80828476
所属: 国立情報学研究所, アーキテクチャ科学研究系, 准教授
キーワード: プログ

In [6]:
soup

<!DOCTYPE html>

<html>
<head>
<!-- meta -->
<meta charset="utf-8"/>
<meta content="no-cache" http-equiv="Pragma">
<meta content="no-cache" http-equiv="Cache-Control"/>
<meta content="no-store" http-equiv="Cache-Control"/>
<meta content="must-revalidate" http-equiv="Cache-Control"/>
<meta content="0" http-equiv="Expires"/>
<meta content="NONE" name="ROBOTS"/>
<meta content="NOINDEX,NOFOLLOW" name="ROBOTS"/>
<meta content="NOARCHIVE" name="ROBOTS"/>
<link href="/static/images/favicon_kakenhi.ico" rel="shortcut icon" type="image/vnd.microsoft.ico"/>
<link href="/static/images/favicon_kakenhi.ico" rel="icon" type="image/vnd.microsoft.ico"/>
<!-- Bootstrap core CSS -->
<link href="/static/css/bootstrap-modal.css?1691548633" id="switch_style" rel="stylesheet"/>
<!-- Custom styles for this template -->
<link href="/static/css/chosentree.css?1697515897" rel="stylesheet"/>
<link href="/static/css/treeselect.css?1697515897" rel="stylesheet"/>
<link href="/static/css/cinii.css?1735186047" rel="s

In [2]:
!pip list

Package                                  Version
---------------------------------------- -----------
absl-py                                  2.1.0
aiohttp                                  3.9.4
aiosignal                                1.3.1
annotated-types                          0.6.0
anyio                                    4.3.0
appnope                                  0.1.4
asgiref                                  3.8.1
asttokens                                2.4.1
astunparse                               1.6.3
attrs                                    23.2.0
backoff                                  2.2.1
bcrypt                                   4.1.2
build                                    1.2.1
cachetools                               5.3.3
certifi                                  2025.1.31
charset-normalizer                       3.4.1
chroma-hnswlib                           0.7.3
chromadb                                 0.4.24
click                                    8.1.7